# Ranked Retrieval Coursework by RahmanMT


## Introduction
This ranked retrival process uses tf_idf to score every term_document pair. 
breif overview:
1. Index the documents' terms and create a postings list containing every document terms and it's tf_idf score
2. Convert query into a vector and assign every term a tf_idf score
3. Calculate cosine(query,document) for all documents, cosine angle is calculated by a.b/|a|.|b|
4. Order the results in descending order and retrive top n results (default n=10)
5. Display results

## Code

#### Imports

import re
import os
import math
import numpy as np
import operator
from collections import OrderedDict

#### Create class and class variables

In [ ]:
class Retrieval:
    documents = []
    document_frequency = {}
    PATH_TO_DOCS = "bbcsport/docs/"

The `documents` list will contain the strings for all the documents, this is to reduce the number of read operations
The `document_frequency` dict will contain all the terms and it's corresponding inverse document frequency score. Due to repeating terms using a dict will be faster than calculating the document frequency everytime it is required.
The `PATH_TO_DOCS` is just a string representation of the path to the folder containing the documents. 

#### Helper functions

In [ ]:
def read_file(path, docid):       
    files = sorted(os.listdir(path))
    f = open(os.path.join(path, files[docid]), 'r', encoding='latin-1')
    s = f.read()
    f.close()
    return s.lower()

Returns the requested document in a string format

In [ ]:
def tokenize(string):             

Tokenizes the input string by normalising and removing duplicates
Normalize by removing capitals and removing commas an delimiters

In [ ]:
DELIM = '[ \n\t0123456789;:.,/\(\)\"\'-]+'
tokens = re.split(DELIM, string.lower())

Removes any empty strings

In [ ]:
if '' in tokens:
    tokens.remove('')

Remove duplicates and return 

In [ ]:
return list(dict.fromkeys(tokens))

#### Initialize documents and document document_frequency

In [ ]:
def initialize_documents(self, path):        
    N = len(sorted(os.listdir(path)))
    for docID in range(N):
        s = self.read_file(path, docID)
        self.documents.append(s)            

Reads all the documents and stores them in the list

In [ ]:
def initialize_doc_freq(self, N):
    for docID in range(N):
        s = self.documents[docID]
        tokens = self.tokenize(s)
        for t in tokens:
            self.calculate_doc_freq(t, N)

Loops through all the documents, for each document it tokenises the terms. Then for each token it calculates the document frequency score and stores the token and score in the `document_frequency` dict